In [34]:
import pandas as pd
import pandas_profiling as pdp
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder

In [40]:
df = pd.read_csv("output/suumo.csv",encoding="utf-16")

In [41]:
i = 1
for r in ["立地1","立地2","立地3"]:
    locals()["sp" + str(i)] = df[r].str.split(" 歩",expand=True)
    locals()["sp" + str(i)].columns = [r+"1",r+"2"]
    i += 1
    
sp4 = df['敷/礼/保証/敷引,償却'].str.split('/', expand=True)
sp4.columns = ['敷金', '礼金']

In [42]:
df = pd.concat([df,sp1,sp2,sp3,sp4], axis=1)
df.drop(['立地1','立地2','立地3','敷/礼/保証/敷引,償却'], axis=1, inplace=True)

In [43]:
df = df.dropna(subset=["賃料料"])

In [44]:
df["賃料料"] = df["賃料料"].str.replace(u"万円", u"")
df["敷金"] = df["賃料料"].str.replace(u"万円", u"")
df["礼金"] = df["礼金"].str.replace(u"万円", u"")
df["管理費"] = df["管理費"].str.replace(u"円",u"")
df["築年数"] = df["築年数"].str.replace(u"新築",u"0")
df["築年数"] = df["築年数"].str.replace(u"99年以上",u"0")
df["築年数"] = df["築年数"].str.replace(u"築",u"")
df["築年数"] = df["築年数"].str.replace(u"年",u"")
df["専有面積"] = df["専有面積"].str.replace(u"m",u"")
df["立地12"] = df["立地12"].str.replace(u"分",u"")
df["立地22"] = df["立地22"].str.replace(u"分",u"")
df["立地32"] = df["立地32"].str.replace(u"分",u"")

In [45]:
for x in ["管理費","敷金","礼金"]:
    df[x] = df[x].replace("-",0)

In [46]:
i = 5
j = 1
for r in ["立地11","立地21","立地31"]:
    locals()["sp"+str(i)] = df[r].str.split("/",expand=True)
    locals()["sp"+str(i)].columns = ["路線"+str(j),"駅"+str(j)]
    locals()["sp"+str(i)]["徒歩"+str(j)] = df[r[:-1]+"2"]
    i += 1
    j += 1

In [47]:
df = pd.concat([df, sp5, sp6, sp7], axis=1)

df.drop(['立地11','立地12','立地21','立地22','立地31','立地32'], axis=1, inplace=True)

In [48]:
for x in ["管理費","賃料料","敷金","礼金","築年数","専有面積","徒歩1","徒歩2","徒歩3"]:
    df[x] = pd.to_numeric(df[x])
    
for x in ["賃料料","敷金","礼金"]:
    df[x] = df[x] * 10000

In [49]:
sp8 = df['階層'].str.split('-', expand=True)
sp8.columns = ['階1', '階2']
sp8['階1'].str.encode('cp932')
sp8['階1'] = sp8['階1'].str.replace(u'階', u'')
sp8['階1'] = sp8['階1'].str.replace(u'B', u'-')
sp8['階1'] = sp8['階1'].str.replace(u'M', u'')
sp8['階1'] = pd.to_numeric(sp8['階1'])

In [50]:
df = pd.concat([df, sp8], axis=1)

In [51]:
df['建物の高さ'] = df['建物の高さ'].str.replace(u'地下1地上', u'')
df['建物の高さ'] = df['建物の高さ'].str.replace(u'地下2地上', u'')
df['建物の高さ'] = df['建物の高さ'].str.replace(u'地下3地上', u'')
df['建物の高さ'] = df['建物の高さ'].str.replace(u'地下4地上', u'')
df['建物の高さ'] = df['建物の高さ'].str.replace(u'地下5地上', u'')
df['建物の高さ'] = df['建物の高さ'].str.replace(u'地下6地上', u'')
df['建物の高さ'] = df['建物の高さ'].str.replace(u'地下7地上', u'')
df['建物の高さ'] = df['建物の高さ'].str.replace(u'地下8地上', u'')
df['建物の高さ'] = df['建物の高さ'].str.replace(u'地下9地上', u'')
df['建物の高さ'] = df['建物の高さ'].str.replace(u'平屋', u'1')
df['建物の高さ'] = df['建物の高さ'].str.replace(u'階建', u'')

In [52]:
df['建物の高さ'] = pd.to_numeric(df['建物の高さ'])

In [53]:
df = df.reset_index(drop=True)

In [54]:
df["間取りDK"] = 0
df["間取りK"] = 0
df["間取りL"] = 0
df["間取りS"] = 0
df["間取り"] = df["間取り"].str.replace(u"ワンルーム",u"1")

In [55]:
def madori(x):
    for i in range(len(df)):
        if x in df["間取り"][i]:
            df.loc[i,"間取り"+x] = 1
    df["間取り"] = df["間取り"].str.replace(x,u"")

In [56]:
for x in ["DK","K","L","S"]:
    madori(x)

In [57]:
df["間取り"] = pd.to_numeric(df["間取り"])

In [58]:
sp9 = df['住所'].str.split('区', expand=True)
sp9.columns = ['区', '市町村']
sp9['区'] = sp9['区'] + '区'
sp9['区'] = sp9['区'].str.replace('東京都','')
df = pd.concat([df, sp9], axis=1)

In [59]:
df[['路線1','路線2','路線3', '駅1', '駅2','駅3','市町村']] = df[['路線1','路線2','路線3', '駅1', '駅2','駅3','市町村']].fillna("NAN")

In [60]:
oe = preprocessing.OrdinalEncoder()
df[['路線1','路線2','路線3', '駅1', '駅2','駅3','市町村']] = oe.fit_transform(df[['路線1','路線2','路線3', '駅1', '駅2','駅3','市町村']].values) 

In [61]:
df['賃料料+管理費'] = df['賃料料'] + df['管理費']

In [62]:
#上限設定
df = df[df["賃料料+管理費"]<150000]

In [63]:
df = df[["マンション名",'賃料料+管理費', '築年数', '建物の高さ', '階1',
       '専有面積','路線1','路線2','路線3', '駅1', '駅2','駅3','徒歩1', '徒歩2','徒歩3','間取り', '間取りDK', '間取りK', '間取りL', '間取りS',
       '市町村','詳細URL']]

df.columns = ['name','real_rent','age', 'height', 'level','area', 'route_1','route_2','route_3','station_1','station_2','station_3','distance_1','distance_2','distance_3','rooms','DK','K','L','S','address',"url"]

In [64]:
df["per_area"] = df["area"]/df["rooms"]
df["height_level"] = df["height"]*df["level"]
df["area_height_level"] = df["area"]*df["height_level"]
df["distance_staion_1"] = df["station_1"]*df["distance_1"]
df["per_area"] = df["area"]/df["rooms"]
df["height_level"] = df["height"]*df["level"]
df["area_height_level"] = df["area"]*df["height_level"]
df["distance_staion_1"] = df["station_1"]*df["distance_1"]

In [65]:
df

,name,real_rent,age,height,level,area,route_1,route_2,route_3,station_1,...,DK,K,L,S,address,url,per_area,height_level,area_height_level,distance_staion_1
36,メゾンドアーク南大井,109000.0,0,13,3.0,26.972,1.0,2.0,11.0,22.0,...,0,1,0,0,25.0,https://suumo.jp/chintai/jnc_000055218472/?bc=...,26.972,39.0,1051.908,154.0
37,メゾンドアーク南大井,109000.0,0,13,3.0,26.972,1.0,2.0,11.0,22.0,...,0,1,0,0,25.0,https://suumo.jp/chintai/jnc_000055243806/?bc=...,26.972,39.0,1051.908,154.0
38,メゾンドアーク南大井,109000.0,0,13,4.0,26.972,1.0,2.0,11.0,22.0,...,0,1,0,0,25.0,https://suumo.jp/chintai/jnc_000055281890/?bc=...,26.972,52.0,1402.544,154.0
39,メゾンドアーク南大井,109000.0,0,13,4.0,26.972,1.0,2.0,11.0,22.0,...,0,1,0,0,25.0,https://suumo.jp/chintai/jnc_000055281887/?bc=...,26.972,52.0,1402.544,154.0
40,メゾンドアーク南大井,110000.0,0,13,3.0,27.842,1.0,2.0,11.0,22.0,...,0,1,0,0,25.0,https://suumo.jp/chintai/jnc_000055281884/?bc=...,27.842,39.0,1085.838,154.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,エルフォルテ品川サウスシティ,110500.0,4,9,5.0,25.952,8.0,2.0,2.0,7.0,...,0,1,0,0,19.0,https://suumo.jp/chintai/jnc_000056559127/?bc=...,25.952,45.0,1167.840,70.0
1896,パークスクエア戸越銀座,93000.0,18,3,2.0,25.022,4.0,10.0,6.0,15.0,...,0,1,0,0,47.0,https://suumo.jp/chintai/jnc_000057074536/?bc=...,25.022,6.0,150.132,60.0
1897,ブリリアタワー品川シーサイド,120000.0,14,22,3.0,30.192,8.0,1.0,2.0,7.0,...,0,1,0,0,69.0,https://suumo.jp/chintai/jnc_000057224210/?bc=...,30.192,66.0,1992.672,140.0
1901,京急本線 立会川駅 15階建 築4年,88000.0,4,15,4.0,22.742,1.0,11.0,5.0,22.0,...,0,0,0,0,25.0,https://suumo.jp/chintai/jnc_000044124294/?bc=...,22.742,60.0,1364.520,176.0


In [66]:
df.to_csv('output/cleaned.csv', encoding='utf-16', header=True, index=False)

In [67]:
# pdp.ProfileReport(df).to_file(output_file="eda.html")